In [1]:
import numpy as np
import pandas as pd
import gzip
import json

from pprint import pprint

In [2]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
turkish_stopwords = stopwords.words('turkish')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Osama\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
train_classification_df = pd.read_csv("/Users/Osama/Downloads/CS412PROJ/train-classification.csv",)
train_classification_df = train_classification_df.rename(columns={'Unnamed: 0': 'user_id', 'label': 'category'})

# Unifying labels
train_classification_df["category"] = train_classification_df["category"].apply(str.lower)
username2_category = train_classification_df.set_index("user_id").to_dict()["category"]

In [4]:
train_classification_df.groupby("category").count()

,user_id
category,
art,191
entertainment,323
fashion,299
food,511
gaming,13
health and lifestyle,503
mom and children,149
sports,113
tech,346


In [5]:
username2_category["kod8net"]

'tech'

In [6]:
'''train_classification_df = pd.read_csv("/Users/Osama/Downloads/CS412PROJ/train-classification.csv",)
train_classification_df = train_classification_df.rename(columns={'Unnamed: 0': 'user_id', 'label': 'category'})

# Unifying labels
train_classification_df["category"] = train_classification_df["category"].apply(str.lower)
username2_category = train_classification_df.set_index("user_id").to_dict()["category"]




# Load the additional CSV file
additional_data_path = "/Users/Osama/Downloads/CS412PROJ/annotated_users_CS412-2753ef4cf74e.csv"
additional_data_df = pd.read_csv(additional_data_path)

# Extract and rename the relevant columns
additional_data_df = additional_data_df[['Unnamed: 0', 'influencerCategory']].rename(columns={
    'Unnamed: 0': 'user_id', 
    'influencerCategory': 'category'
})

additional_data_df = additional_data_df.dropna()



# Convert the 'category' column to string type and apply .lower(), handling any NaN or unexpected values
additional_data_df['category'] = additional_data_df['category'].astype(str).fillna('').apply(str.lower)

# Append the new data to the original train_classification_df
train_classification_df = pd.concat([train_classification_df, additional_data_df], ignore_index=True)

# Update the username2_category dictionary with the new data
username2_category.update(additional_data_df.set_index("user_id").to_dict()["category"])

# Check the updated data by viewing the first few rows
print(train_classification_df.head())

# Re-check the category distribution
train_classification_df.groupby("category").count()'''


'train_classification_df = pd.read_csv("/Users/Osama/Downloads/CS412PROJ/train-classification.csv",)\ntrain_classification_df = train_classification_df.rename(columns={\'Unnamed: 0\': \'user_id\', \'label\': \'category\'})\n\n# Unifying labels\ntrain_classification_df["category"] = train_classification_df["category"].apply(str.lower)\nusername2_category = train_classification_df.set_index("user_id").to_dict()["category"]\n\n\n\n\n# Load the additional CSV file\nadditional_data_path = "/Users/Osama/Downloads/CS412PROJ/annotated_users_CS412-2753ef4cf74e.csv"\nadditional_data_df = pd.read_csv(additional_data_path)\n\n# Extract and rename the relevant columns\nadditional_data_df = additional_data_df[[\'Unnamed: 0\', \'influencerCategory\']].rename(columns={\n    \'Unnamed: 0\': \'user_id\', \n    \'influencerCategory\': \'category\'\n})\n\nadditional_data_df = additional_data_df.dropna()\n\n\n\n# Convert the \'category\' column to string type and apply .lower(), handling any NaN or unexpec

In [7]:
train_data_path = "/Users/Osama/Downloads/CS412PROJ/training-dataset.jsonl.gz"

username2posts_train = dict()
username2profile_train = dict()

username2posts_test = dict()
username2profile_test = dict()


with gzip.open(train_data_path, "rt") as fh:
  for line in fh:
    sample = json.loads(line)

    profile = sample["profile"]
    username = profile["username"]
    if username in username2_category:
      # train data info
      username2posts_train[username] = sample["posts"]
      username2profile_train[username] = profile


    else:
      # it is test data info
      username2posts_test[username] = sample["posts"]
      username2profile_test[username] = profile

In [8]:
train_profile_df = pd.DataFrame(username2profile_train).T.reset_index(drop=True)
test_profile_df = pd.DataFrame(username2profile_test).T.reset_index(drop=True)

train_profile_df.shape

(2741, 44)

In [9]:
test_profile_df.head(2)

,username,id,full_name,biography,category_name,post_count,follower_count,following_count,is_business_account,is_private,...,business_category_name,overall_category_name,category_enum,is_verified_by_mv4b,is_regulated_c18,profile_pic_url,should_show_category,should_show_public_contacts,show_account_transparency_details,profile_picture_base64
0,beyazyakaliyiz,8634457436,Selam Beyaz Yakalı,Beyaz yakalıların dünyasına hoşgeldiniz 😀😀😀,Personal blog,None,1265,665,True,False,...,None,None,PERSONAL_BLOG,False,False,https://instagram.fist6-1.fna.fbcdn.net/v/t51....,True,True,True,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
1,totalenergies_istasyonlari,7066643793,TotalEnergies İstasyonları,TotalEnergies İstasyonları resmi Instagram hes...,Energy Company,None,28025,4,True,False,...,None,None,ENERGY_COMPANY,False,False,https://instagram.fsaw2-1.fna.fbcdn.net/v/t51....,True,True,True,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...


In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import emoji


def preprocess_text(text: str):
    

    # lower casing Turkish Text, Don't use str.lower :)
    text = text.casefold()

    #text = emoji.demojize(text, delimiters=(" ", " "))

    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

    # Remove special characters and punctuation
    # HERE THE EMOJIS stuff are being removed, you may want to keep them :D
    text = re.sub(r'[^a-zçğıöşü0-9\s#@]', '', text)

    # Remove numbers
    text = re.sub(r'\d+', '', text)

    # Remove extra whitespaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text


corpus = []

# to keep the label order
train_usernames = []

for username, posts in username2posts_train.items():
  train_usernames.append(username)

  # aggregating the posts per user
  cleaned_captions = []
  for post in posts:
    post_caption = post.get("caption", "")
    if post_caption is None:
      continue

    post_caption = preprocess_text(post_caption)

    if post_caption != "":
      cleaned_captions.append(post_caption)


  # joining the posts of each user with a \n
  user_post_captions = "\n".join(cleaned_captions)
  corpus.append(user_post_captions)



#custom_stopwords = list(set(turkish_stopwords).union({
 #   'the', 'and', 'with', 'for', 'you', 'to', 'of', 'in', 'our', 'your', 'is', 'are','bir'
#}))
vectorizer = TfidfVectorizer(stop_words=turkish_stopwords, max_features=10000,min_df=10,ngram_range=(1, 3))

# fit the vectorizer
vectorizer.fit(corpus)

# transform the data into vectors
x_post_train = vectorizer.transform(corpus)
y_train = [username2_category.get(uname, "NA") for uname in train_usernames]
feature_names = vectorizer.get_feature_names_out()

# Inspect the frequency of each word
df_tfidf = pd.DataFrame(x_post_train.toarray(), columns=feature_names)

# Show the most frequent words (words in many posts)
print(df_tfidf.sum().sort_values(ascending=False).head(30))




test_usernames = []
test_corpus = []
for username, posts in username2posts_test.items():
  test_usernames.append(username)
  # aggregating the posts per user
  cleaned_captions = []
  for post in posts:
    post_caption = post.get("caption", "")
    if post_caption is None:
      continue

    post_caption = preprocess_text(post_caption)

    if post_caption != "":
      cleaned_captions.append(post_caption)

  user_post_captions = "\n".join(cleaned_captions)
  test_corpus.append(user_post_captions)


# Just transforming! No Fitting!!!!!
x_post_test = vectorizer.transform(test_corpus)

bir            192.474290
the            106.220712
and             65.960927
to              59.345598
of              59.225437
olsun           56.723444
yeni            46.531255
istanbul        46.083638
in              44.908546
olarak          43.520030
olan            41.210234
with            39.918309
kutlu           39.892111
kutlu olsun     39.315072
iyi             37.609096
güzel           37.207437
kadar           36.265992
you             35.986349
for             34.862311
özel            32.566009
bilgi           32.489645
devam           32.122644
var             31.813758
our             31.544601
kasım           31.361084
günü            31.188695
birlikte        30.770312
büyük           30.195825
ilk             29.995598
teşekkür        29.513940
dtype: float64


In [11]:
# Making sure everything is fine
assert y_train.count("NA") == 0


In [12]:
feature_names = vectorizer.get_feature_names_out()
feature_names

array(['ab', 'abant', 'abd', ..., 'şıklık', 'şıklığı', 'şımartın'],
      dtype=object)

In [13]:
df_tfidf = pd.DataFrame(x_post_train.toarray(), columns=feature_names)
df_tfidf.head(2)

,ab,abant,abd,abdulkadir,abdullah,abi,abiye,abone,about,about the,...,şöyle,şükran,şükranla,şükranlarımızı,şükür,şık,şık bir,şıklık,şıklığı,şımartın
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.042156,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [14]:
df_tfidf.shape

(2741, 10000)

In [15]:
print("Shape of df_tfidf:", df_tfidf.shape)
print("Length of y_train:", len(y_train))

Shape of df_tfidf: (2741, 10000)
Length of y_train: 2741


In [16]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(df_tfidf, y_train, test_size=0.2, stratify=y_train,random_state=42)

In [17]:
x_train.shape

(2192, 10000)

In [18]:
x_val.shape

(549, 10000)

In [19]:
#@title Test Data
test_data_path = "/Users/Osama/Downloads/CS412PROJ/test-classification-round1.dat"

with open(test_data_path, "rt") as fh:
    for i, line in enumerate(fh):
        print(line.strip())
        if i >= 4:  # Stop after 5 lines
            break

print("*****")

test_unames = []
with open(test_data_path, "rt") as fh:
  for line in fh:
    test_unames.append(line.strip())

print(test_unames[:5])

ozhotelstr
elleturkiye
sozerinsaatorhangazi
sanliurfapiazzaavym
rusanozden
*****
['ozhotelstr', 'elleturkiye', 'sozerinsaatorhangazi', 'sanliurfapiazzaavym', 'rusanozden']


In [20]:
x_test = []

for uname in test_unames:
  try:
    index = test_usernames.index(uname)
    x_test.append(x_post_test[index].toarray()[0])
  except Exception as e:
    try:
      index = train_usernames.index(uname)
      x_test.append(x_post_train[index].toarray()[0])
    except Exception as e:
      print(uname)


test_unames.remove("screenname")

screenname


In [21]:
df_test = pd.DataFrame(np.array(x_test), columns=feature_names)
df_test.head(2)

,ab,abant,abd,abdulkadir,abdullah,abi,abiye,abone,about,about the,...,şöyle,şükran,şükranla,şükranlarımızı,şükür,şık,şık bir,şıklık,şıklığı,şımartın
0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,0.0,0.0,0.007865,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.024666,0.0,0.0,0.0,0.0,0.010521,0.0,0.0,0.0,0.0


In [22]:
def predict_like_count(username, current_post=None):
  def get_avg_like_count(posts:list):
    total = 0.
    for post in posts:
      if current_post is not None and post["id"] == current_post["id"]:
        continue

      like_count = post.get("like_count", 0)
      if like_count is None:
        like_count = 0
      total += like_count

    if len(posts) == 0:
      return 0.

    return total / len(posts)

  if username in username2posts_train:
    return get_avg_like_count(username2posts_train[username])
  elif username in username2posts_test:
    return get_avg_like_count(username2posts_test[username])
  else:
    print(f"No data available for {username}")
    return -1

In [23]:
def log_mse_like_counts(y_true, y_pred):
  """
  Calculate the Log Mean Squared Error (Log MSE) for like counts (log(like_count + 1)).

  Parameters:
  - y_true: array-like, actual like counts
  - y_pred: array-like, predicted like counts

  Returns:
  - log_mse: float, Log Mean Squared Error
  """
  # Ensure inputs are numpy arrays
  y_true = np.array(y_true)
  y_pred = np.array(y_pred)

  # Log transformation: log(like_count + 1)
  log_y_true = np.log1p(y_true)
  log_y_pred = np.log1p(y_pred)

  # Compute squared errors
  squared_errors = (log_y_true - log_y_pred) ** 2

  # Return the mean of squared errors
  return np.mean(squared_errors)

In [33]:
#@title Train Dataset evaluation

y_like_count_train_true = []
y_like_count_train_pred = []
for uname, posts in username2posts_train.items():
  for post in posts:
    pred_val = predict_like_count(uname, post)
    true_val = post.get("like_count", 0)
    if true_val is None:
      true_val = 0

    y_like_count_train_true.append(true_val)
    y_like_count_train_pred.append(pred_val)

print(f"Log MSE Train= {log_mse_like_counts(y_like_count_train_true, y_like_count_train_pred)}")

Log MSE Train= 1.2271047744059362


In [34]:
#@title Test Dataset

path = "/Users/Osama/Downloads/CS412PROJ/test-regression-round1.jsonl"
output_path = "/Users/Osama/Downloads/CS412PROJ/testy-regression-round1.jsonl"

to_predict_like_counts_usernames = []
output_list = []
with open(path, "rt") as fh:
  for line in fh:
    sample = json.loads(line)

    # let's predict
    pred_val = predict_like_count(sample["username"])
    sample["like_count"] = int(pred_val)
    output_list.append(sample)

with open(output_path, "wt") as of:
  json.dump(output_list, of)




In [45]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor


# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

# Initialize the regressor
rf = RandomForestRegressor(random_state=42)

# Perform grid search
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', verbose=2, n_jobs=-1)
grid_search.fit(x_train, y_train)

# Best parameters and best model
print("Best Parameters:", grid_search.best_params_)
best_model = grid_search.best_estimator_


Fitting 5 folds for each of 324 candidates, totalling 1620 fits


C:\Users\Osama\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
540 fits failed out of a total of 1620.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
173 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Osama\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Osama\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-

Best Parameters: {'max_depth': 30, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 50}


In [49]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# Prepare the target variable: Average like count per user
user_avg_likes = {}
for username, posts in username2posts_train.items():
    avg_likes = np.mean([post.get("like_count", 0) or 0 for post in posts])
    user_avg_likes[username] = avg_likes

# Create the target array
y_like_counts = [user_avg_likes[uname] for uname in train_usernames]
y_like_counts = [0 if np.isnan(val) else val for val in y_like_counts]

# Train-test split
x_train, x_val, y_train, y_val = train_test_split(
    x_post_train, y_like_counts, test_size=0.2, random_state=42
)
# Train a Random Forest Regressor
regressor = RandomForestRegressor(n_estimators=100, random_state=42)
regressor.fit(x_train, y_train)

# Evaluate the model
y_pred_val = regressor.predict(x_val)
if np.isnan(y_pred_val).any():
    print("NaN values found in predictions!")
else:
    mse = mean_squared_error(y_val, y_pred_val)
    log_mse = log_mse_like_counts(y_val, y_pred_val)
    print(f"MSE on validation set: {mse}")
    print(f"Log MSE on validation set: {log_mse}")

'''
mse = mean_squared_error(y_val, y_pred_val)
log_mse = log_mse_like_counts(y_val, y_pred_val)

print(f"MSE on validation set: {mse}")
print(f"Log MSE on validation set: {log_mse}")
'''
# Predict like counts for the test dataset
y_test_pred = regressor.predict(x_post_test)

# Create a DataFrame for test predictions
test_predictions = pd.DataFrame({
    "username": test_usernames,
    "predicted_like_count": y_test_pred
})
print(test_predictions.head())


C:\Users\Osama\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\_core\fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Osama\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\_core\_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


KeyboardInterrupt: 

In [47]:
print(test_predictions.head(50))

                      username  predicted_like_count
0               beyazyakaliyiz          15745.513062
1   totalenergies_istasyonlari           2613.090460
2                  konforyatak            612.966028
3                     ht_kulup           2220.125502
4                    ajansspor            613.633731
5         yusufelibelediyesi08           4725.989184
6                     4bros.tr           1317.076829
7              groundy.kadikoy            618.112526
8                drtubagunebak           7327.731370
9                   nihatcan11            757.510589
10               mustafaaakcay            608.791499
11                  mtmofamily          22334.793104
12                 bonitoperde           1038.041119
13     balkanlardangelenlezzet            587.165343
14      pedagoganne__gulozturk          17995.796036
15                      iktm34           3006.868508
16                    yudumyag           2482.609961
17      bulentozdemir.edebiyat            989.

In [48]:
log_errors = np.log1p(y_val) - np.log1p(y_pred_val)  # Use np.log1p to handle zero values gracefully

# Compute the absolute log errors
abs_log_errors = np.abs(log_errors)

# Sum of log errors
sum_log_errors = np.sum(abs_log_errors)

print(f"Sum of Log Errors: {sum_log_errors}")

Sum of Log Errors: 1726.152889784193


In [39]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# Prepare the target variable: Average like count per user
user_avg_likes = {}
for username, posts in username2posts_train.items():
    avg_likes = np.mean([post.get("like_count", 0) or 0 for post in posts])
    user_avg_likes[username] = avg_likes

# Create the target array
y_like_counts = [user_avg_likes[uname] for uname in train_usernames]
y_like_counts = [0 if np.isnan(val) else val for val in y_like_counts]
# Train-test split
x_train, x_val, y_train, y_val = train_test_split(
    x_post_train, y_like_counts, test_size=0.2, random_state=42
)

# Train a Linear Regression model
regressor = LinearRegression()
regressor.fit(x_train, y_train)

# Evaluate the model
y_pred_val = regressor.predict(x_val)
if np.isnan(y_pred_val).any():
    print("NaN values found in predictions!")
else:
    mse = mean_squared_error(y_val, y_pred_val)
    log_mse = log_mse_like_counts(y_val, y_pred_val)
    print(f"MSE on validation set: {mse}")
    print(f"Log MSE on validation set: {log_mse}")
mse = mean_squared_error(y_val, y_pred_val)
log_mse = log_mse_like_counts(y_val, y_pred_val)

print(f"MSE on validation set: {mse}")
print(f"Log MSE on validation set: {log_mse}")

# Predict like counts for the test dataset
y_test_pred = regressor.predict(x_post_test)

# Create a DataFrame for test predictions
test_predictions = pd.DataFrame({
    "username": test_usernames,
    "predicted_like_count": y_test_pred
})
print(test_predictions.head())


C:\Users\Osama\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\_core\fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Osama\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\_core\_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


MSE on validation set: 2264039389.2164493
Log MSE on validation set: nan
MSE on validation set: 2264039389.2164493
Log MSE on validation set: nan
                     username  predicted_like_count
0              beyazyakaliyiz          30795.692201
1  totalenergies_istasyonlari         -10663.118432
2                 konforyatak           3044.176534
3                    ht_kulup           7806.627423
4                   ajansspor         -31180.186806


C:\Users\Osama\AppData\Local\Temp\ipykernel_12760\3326594812.py:18: RuntimeWarning: invalid value encountered in log1p
  log_y_pred = np.log1p(y_pred)
C:\Users\Osama\AppData\Local\Temp\ipykernel_12760\3326594812.py:18: RuntimeWarning: invalid value encountered in log1p
  log_y_pred = np.log1p(y_pred)


In [36]:
# Check for NaN in user_avg_likes
for username, posts in username2posts_train.items():
    avg_likes = np.mean([post.get("like_count", 0) or 0 for post in posts])
    if np.isnan(avg_likes):
        print(f"NaN detected for user: {username}, posts: {posts}")
    user_avg_likes[username] = avg_likes


NameError: name 'user_avg_likes' is not defined

In [28]:
'''# Calculate log errors
log_errors = np.abs(y_val_reg - y_val_pred_reg)

# Sum of log errors
sum_log_errors = np.sum(log_errors)

print("Sum of Log Errors:", sum_log_errors)
'''

NameError: name 'y_val_pred_reg' is not defined